<a id="c45"></a>
# <font color="#004D7F"> Implementación del C4.5</font>

En este apartado vamos a explicar la segunda parte de vuestro trabajo con los árboles de decisión: la implementación del C4.5.

Como hemos visto antes, la implementación de `scikit-learn` no tiene en cuenta las variables categóricas o discretas. Por eso, nosotros vamos a hacer una implementación del C4.5 que tenga en cuenta tanto las variables categóricas como las continuas. 

Esta implementación va a consistir en una versión simplificada del algoritmo. Al tratarse de un problema de clasificación, utilizaremos la entropía (en vez de Gini) para calcular la ganancia y no será obligatorio realizar la poda. Si será obligatorio implementar, por lo menos, el hiperparámetro de profundida máxima. La implementación del Gini, así como la poda o cualquier otro hiperparámetro como el `min_samples_leaf` visto en `scikit-learn` serán partes opcionales de la práctica.

Vamos a seguir la estructura de los algoritmos de SciKit, por lo que deberemos implementar una clase `C45` que herede de `BaseEstimator` y que tenga las funciones `__init__`,  `fit` y `predict`. Al final se deberá comparar el comportamiento del algoritmo implementado con los resultados obtenidos de la versión de `scikit-learn`.

In [24]:
import numpy as np
import pandas as pd
import math
import copy

from sklearn import tree
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

import graphviz 
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
df_rain = pd.read_csv('weatherAUS.csv')
df_rain

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [26]:
# Variables continuas
df_rain['MinTemp'] = df_rain['MinTemp'].fillna(df_rain['MinTemp'].mean())
df_rain['MaxTemp'] = df_rain['MaxTemp'].fillna(df_rain['MaxTemp'].mean())
df_rain['Rainfall'] = df_rain['Rainfall'].fillna(df_rain['Rainfall'].mean())
df_rain['WindGustSpeed'] = df_rain['WindGustSpeed'].fillna(df_rain['WindGustSpeed'].mean())
df_rain['WindSpeed9am'] = df_rain['WindSpeed9am'].fillna(df_rain['WindSpeed9am'].mean())
df_rain['WindSpeed3pm'] = df_rain['WindSpeed3pm'].fillna(df_rain['WindSpeed3pm'].mean())
df_rain['Humidity9am'] = df_rain['Humidity9am'].fillna(df_rain['Humidity9am'].mean())
df_rain['Humidity3pm'] = df_rain['Humidity3pm'].fillna(df_rain['Humidity3pm'].mean())
df_rain['Pressure9am'] = df_rain['Pressure9am'].fillna(df_rain['Pressure9am'].mean())
df_rain['Pressure3pm'] = df_rain['Pressure3pm'].fillna(df_rain['Pressure3pm'].mean())
df_rain['Cloud9am'] = df_rain['Cloud9am'].fillna(df_rain['Cloud9am'].mean())
df_rain['Cloud3pm'] = df_rain['Cloud3pm'].fillna(df_rain['Cloud3pm'].mean())
df_rain['Temp9am'] = df_rain['Temp9am'].fillna(df_rain['Temp9am'].mean())
df_rain['Temp3pm'] = df_rain['Temp3pm'].fillna(df_rain['Temp3pm'].mean())

# Variables categóricas
df_rain['Location'].replace(np.nan, 'Canberra', inplace=True)
df_rain['WindGustDir'].replace(np.nan, 'W', inplace=True)
df_rain['WindDir9am'].replace(np.nan, 'N', inplace=True)
df_rain['WindDir3pm'].replace(np.nan, 'SE', inplace=True)
df_rain['RainToday'].replace(np.nan, 'No', inplace=True)
df_rain['RainTomorrow'].replace(np.nan, 'No', inplace=True)

In [27]:
"""
encoder = LabelEncoder()
df_rain['RainTomorrow'] = encoder.fit_transform(df_rain['RainTomorrow'])
df_rain['Location'] = encoder.fit_transform(df_rain['Location'])
df_rain['WindGustDir'] = encoder.fit_transform(df_rain['WindGustDir'])
df_rain['WindDir9am'] = encoder.fit_transform(df_rain['WindDir9am'])
df_rain['WindDir3pm'] = encoder.fit_transform(df_rain['WindDir3pm'])
df_rain['RainToday'] = encoder.fit_transform(df_rain['RainToday'])
"""

"\nencoder = LabelEncoder()\ndf_rain['RainTomorrow'] = encoder.fit_transform(df_rain['RainTomorrow'])\ndf_rain['Location'] = encoder.fit_transform(df_rain['Location'])\ndf_rain['WindGustDir'] = encoder.fit_transform(df_rain['WindGustDir'])\ndf_rain['WindDir9am'] = encoder.fit_transform(df_rain['WindDir9am'])\ndf_rain['WindDir3pm'] = encoder.fit_transform(df_rain['WindDir3pm'])\ndf_rain['RainToday'] = encoder.fit_transform(df_rain['RainToday'])\n"

In [28]:
features = df_rain.columns
attributes = features[1:-1]  
target = features[-1]

In [29]:
X = df_rain[attributes]

In [30]:
y = df_rain[target] 

In [31]:
from sklearn.base import BaseEstimator

# Ejemplo de la estructura básica de un algoritmo que hereda de BaseEstimator siguiendo la estructura de SciKit
class C45(BaseEstimator):
    # constructor
    def __init__(self):
        return

    # siguiendo las guias de scikit-learn disponibles en https://scikit-learn.org/stable/developers/develop.html,
    # creamos las funciones fit para el entrenamiento
    def fit(self, X, y):
        return
        
    # y predict para las predicciones de casos nuevos
    def predict(self, X):
        return
    
    # tambien se da una funcion score que deberá calcular el accuracy
    def score(self, X, y):
        return
        

Sabemos que la forma de trabajar con SciKit puede ser algo restrictiva. Por eso, a continuación os dejamos un esqueleto del árbol de decisión, para que os podáis centrar en el algoritmo y no en los detalles técnicos de SciKit o python. 

Esto es totalmente opcional, el que se encuentre más cómodo programando el algoritmo desde 0 es libre de hacerlo, mientras los resultados sean correctos y por lo menos se mantenga la estructura del `BaseEstimator`

In [32]:
class Tree_c45:
    def __init__(self):
        # primero indicamos si el objeto es hoja o raiz
        self.is_leaf = True
        
        # atributos para el corte cuando el objeto es una raiz
        # indicamos el indice de la variable de corte y los valores
        self.var_index = -1
        self.cut_value = 0
        # los hijos deben ser objetos de Tree_c45
        self.list_of_childs = []
        
        # atributos cuando el Tree_c45 es una hoja
        # indicamos el valor de la clase y en class_count una tupla (casos con ese valor, casos totales en la hoja)
        self.class_value = -1
        self.class_count = 0
        self.valorRegresion = ""
        
        # para el nivel de profundidad en el arbol de la raiz o la hoja
        self.level = -1
        
    def __str__(self):
        output = ''
        if(self.is_leaf):
            if(self.valorRegresion == ""):
                output += 'Class value: ' + str(self.class_value) + '\tCounts: ' + str(self.class_count)
            else:
                output += self.valorRegresion
        else:
            output += 'Feature '+ str(self.var_index)
            for i in range(len(self.list_of_childs)):
                output += '\n'+'\t'*(self.level+1)+str(self.cut_value)+': '+str(self.list_of_childs[i])
            
        return output
    
    # esta funcion nos servira para hacer predicciones y debe ser completada
    def predict(self,x):

        if (self.is_leaf == True):

            return self.class_value

        else:

            columna = self.var_index
            valorDeColumna = x[columna].unique()[0]

            if(isinstance(valorDeColumna, str)):

                if (valorDeColumna == self.cut_value):
                    hijoExpandir = self.list_of_childs[0]

                else:
                    hijoExpandir = self.list_of_childs[1]

            else:
                if (valorDeColumna <= self.cut_value):
                    hijoExpandir = self.list_of_childs[0]

                else:
                    hijoExpandir = self.list_of_childs[1]  

            return hijoExpandir.predict(x)

Ya que en esta entrega teníamos que calcular tanto el `GINI` como la `entropía` para la ganancia de información, necesitamos tener dos métodos para cada uno. El primero calcula el `GINI` o la `entropía` con `dataframes`, y el segundo asume que vas a pasarle un array de `numPy`.

In [33]:
def giniConDataframes(y):
    
        if len(y.value_counts()) == 1:
            return 0

        a = y.value_counts()[0]
        b = y.value_counts()[1]

        total = a + b

        gini = 1 - ((pow((a / total), 2)) + (pow((b / total), 2)))

        return gini

def entropiaConDataframes(y):
    
        if len(y.value_counts()) == 1:
            return 0

        total = y.value_counts()[0] + y.value_counts()[1]

        entropia = (-y.value_counts()[0] / total * np.log2(y.value_counts()[0] / total)) + (-y.value_counts()[1] / total * np.log2(y.value_counts()[1] / total))

        return entropia

def entropiaConNumPy(y, useEncoder):
   
        if useEncoder:
            a = len(y[y == 0.0])
            b = len(y[y == 1.0])
        else:
            a = len(y[y == 'No'])
            b = len(y[y == 'Yes'])

        if a == 0 or b == 0:
            return 0

        total = a + b

        entropia = (-a/ total * np.log2(a / total)) + (-b / total * np.log2(b / total))

        return entropia

def gini(y, useEncoder):

    if useEncoder:
        a = len(y[y == 0.0])
        b = len(y[y == 1.0])
    else:
        a = len(y[y == 'No'])
        b = len(y[y == 'Yes'])

    if a == 0 or b == 0:
        return 0

    total = a + b

    aux =  1 - ((pow((a / total), 2)) + (pow((b / total), 2)))

    return aux

In [34]:
from scipy.stats import entropy
from sklearn.base import BaseEstimator

class C45(BaseEstimator):
    # constructor con la profundidad del arbol, que por defecto es 2 y donde inicializamos el arbol que vamos a aprender
    def __init__(self, max_depth=2):
        self.max_depth = max_depth
        self.tree = Tree_c45()
        self.metodo = 1 # el tipo de metodo, gini o entropia, 1 para entropia 2 para gini
        self.maxRamificacion = {}
        self.limiteRamificacion = 1
        self.useEncoder = False
        self.columnToEncode = 'RainTomorrow'
        self.regresion = False
        
    # siguiendo las guias de scikit-learn, creamos las funciones fit para el entrenamiento
    def fit(self, X, y):

        for columna in X:
            self.maxRamificacion[columna] = 0

        if self.useEncoder:
            encoder = LabelEncoder()
            df_rain[self.columnToEncode] = encoder.fit_transform(df_rain[self.columnToEncode])
            features = df_rain.columns
            target = features[-1]
            y = df_rain[target] 

        # esta será la llamada a la funcion recursiva que aprendera el arbol a partir de los datos
        self.make_cut(X, y, self.tree, 0)
        
    # y predict para las predicciones de casos nuevos
    def predict(self, X):
        return np.array([self.tree.predict(x) for x in X])
    
    # tambien se da una funcion score que calcula el accuracy
    def score(self, X, y):
        return np.sum(self.predict(X)==y)/y.shape[0]
    
    # esta es la funcion principal, encargada de encontrar la variable y su punto de corte que maximice la ganancia en base a la entropia.
    # se basa en que los cortes se van a realizar sobre variables continuas aunque la salida sea discreta
    def make_cut(self, X, y, current_tree, current_depth):

        # esqueleto con la funcionalidad basica

        # antes de realizar ninguna operacion comprobamos si hemos alcanzado el limite del arbol,
        # si es asi, actualizamos el arbol ya que estamos en un nodo hoja 
        # y no hace falta buscar variable ni continuar con las llamadas recursivas
        
        if current_depth >= self.max_depth:
            current_tree.is_leaf = True # Al haber superado la profundidad maxima, es una hoja, por lo que vamos a ver los datos que tiene.
            a = y.value_counts()[0]
            b = y.value_counts()[1]
            if(a > b):
                current_tree.class_value = 0 # Sacamos el valor de la variable que mas se repita
                current_tree.class_count = a # Sacamos la cantidad del valor que mas se ha repetido
                if (self.regresion == True):
                    valor0 = 1 - a / (a + b)
                    current_tree.valorRegresion = ("Valor Regresion = " + str(valor0))
            else:
                current_tree.class_value = 1 # Sacamos el valor de la variable que mas se repita
                current_tree.class_count = b # Sacamos la cantidad del valor que mas se ha repetido
                if (self.regresion == True):
                    valor1 = b / (a + b)
                    current_tree.valorRegresion = ("Valor Regresion = " + str(valor1))
            return

        # Primero, obtenemos la entropia de la clase
        if self.metodo == 1:
            mejorValor = entropiaConDataframes(y)
        else:
            mejorValor = giniConDataframes(y)
        
        # Segundo, deberemos recorrer todas las caracteristicas siguiendo estos pasos
        mejorColumna = "MinTemp"
        puntoCorte = 0
        gananciaMax = 0

        # Para cada columna en las variables predictoras:
        for columna in X:

            # 1. Ordenar los valores de la caracteristica y aplicar a la clase
            aux = pd.DataFrame([X[columna], y])
            aux = aux.transpose()
            aux = aux.sort_values(by = columna)

            # 2. Obtener los puntos de corte en base a la clase ordenada
            puntosCorte = aux[columna].unique()
            puntosCorte = np.array(puntosCorte)

            debug = False
            if debug:
                print(puntosCorte)

            # Pasamos a numpy el dataframe
            aux = aux.to_numpy()
            predictora = []
            objetivo = []

            for n in range(0, len(aux)):
                objetivo.append(aux[n][1])
                predictora.append(aux[n][0])

            objetivo = np.array(objetivo)
            predictora = np.array(predictora)

            debug = False
            if debug:
                print(objetivo)
                print(predictora)

            previous = []
            longitud = len(objetivo)
            mejora = False

            # 3. Para cada corte, calculamos la entropia
            for punto in puntosCorte:

                previous = predictora[predictora < punto]
                
                lenPrevious = len(previous)

                objPrev = objetivo[:lenPrevious]
                objNext = objetivo[lenPrevious:]

                if self.metodo == 1:
                    valor1 = entropiaConNumPy(objPrev, self.useEncoder)
                    valor2 = entropiaConNumPy(objNext, self.useEncoder)
                else:
                    valor1 = gini(objPrev, self.useEncoder)
                    valor2 = gini(objNext, self.useEncoder)

                    if debug:
                        print(str(valor1))
                        print(str(valor2) + "\n")

                total = valor1 * (lenPrevious / longitud) + valor2 * ((longitud - lenPrevious) / longitud)

                debug = False
                if debug:
                    print("Valor 1: " + str(valor1))
                    print("Valor 2: " + str(valor2))
                    print("Total: " + str(total))

                # 4. Calculamos la ganancia con la entropia obtenida y la de la clase
                if self.metodo == 1:
                    gananciaInfo = mejorValor - total

                    # 5. Almacenamos la variable y el punto de corte con mayor ganancia             
                    if gananciaInfo > gananciaMax:

                        gananciaMax = gananciaInfo
                        mejora = True
                        mejorValor = total
                        puntoCorte = punto
                        mejorColumna = columna

                        # Calculamos las dos particiones que se van a hacer en la base de datos
                        g1 = X[X[columna] <= punto]
                        g2 = X[X[columna] > punto]

                        gMejor = [g1, g2]
                else:

                    # Almacenamos la variable y el punto de corte con mayor ganancia
                    if total < mejorValor:

                        mejorValor = total;
                        mejora = True
                        puntoCorte = punto
                        mejorColumna = columna

                        # Calculamos las dos particiones que se van a hacer en la base de datos
                        g1 = X[X[columna] <= punto]
                        g2 = X[X[columna] > punto]

                        gMejor = [g1, g2]

            debug = False
            if mejora and debug:
                print("Columna: " + str(columna)) 
                print("Mejor entorpia: " + str(mejorValor))
                print("Punto de corte: " + str(puntoCorte))  
                print("")

        debug = False
        if debug:
            print("Ramificar por columna: " + str(mejorColumna))
            print("Mejor entropia: " + str(mejorValor))
            print("Punto de corte: " + str(puntoCorte))
        
        # Antes de llamar a la funcion recursiva hay que actualizar los valores del arbol
        if debug:
            print("Antes")
            print(gMejor[0])
        
        if(isinstance(gMejor[0][mejorColumna].iloc[0], str)):
            gMejor[0] = gMejor[0].drop(mejorColumna, axis=1)
            gMejor[1] = gMejor[1].drop(mejorColumna, axis=1)

        if debug:
            print("Despues")
            print(gMejor[0])
        
        if (self.maxRamificacion[mejorColumna] < self.limiteRamificacion - 1):
            self.maxRamificacion[mejorColumna] = self.maxRamificacion[mejorColumna] + 1
        elif mejorColumna in gMejor[0]:
                gMejor[0] = gMejor[0].drop(mejorColumna, axis=1)
                gMejor[1] = gMejor[1].drop(mejorColumna, axis=1)
        
        tree1 = Tree_c45()
        tree2 = Tree_c45()

        current_tree.is_leaf = False
        current_tree.level = current_depth
        current_tree.var_index = mejorColumna
        current_tree.cut_value = puntoCorte
        current_tree.list_of_childs = [tree1, tree2]

        # para continuar con el algoritmo hay que hacer una llamada a la funcion
        # de forma recursiva para cada hijo
        for i in range (len(current_tree.list_of_childs)):
            self.make_cut(gMejor[i], y.loc[gMejor[i].index], current_tree.list_of_childs[i], current_tree.level+1)
        
        return

El algoritmo debe implementar por lo menos un hiperparámetro `max_depth` como condición de parada. Seguid el ejemplo mostrado en la función `__init__` del código.

Para comprobar que funciona correctamente, creamos un objeto  que entrenamos con `fit` y del que obtenemos una tasa de aciertos con `score`. Subiremos ejemplos de la salida del algoritmo al campus virtual para que podáis comprobar vuestra implementación.

Para finalizar, hay que comparar los resultados obtenidos por nuestra implementación frente a la implementación de `scikit-learn`. También habrá que comparar que variables son más importantes ahora y obtener las gráficas correspondientes. Este estudio no tiene que ser tan exhaustivo como el de la primera sección.


## Entrega & Pruebas

Las dos primeras ejecuciones del algoritmo que queremos hacer son para un árbol de profundidad 1 usando el método `GINI` para el cálculo de la ganancia de información. Lo importante que tenemos que tener en cuenta en estas dos ejecuciones es que hemos implementado `dos formas distintas` de tratar la `variable objetivo` dentro del árbol. La primera es tratándola como variable `categórica` ('Yes' y 'No') y la segunda como variable `continua` (1 y 0). Esto lo hemos hecho porque el algoritmo es bastante `más eficiente` tratanto números que tratando strings, y se puede comprobar en los distintos tiempos que toman los algoritmos. El tiempo del mismo árbol usando el `encoder` y no usándolo es de 41.5s y de 1m 36.4s, respectivamente. Por lo tanto, hemos decidido usar el encoder en todas las pruebas. Aun usando el encoder, el algoritmo solo codifica la variable objetivo, y no las variables predictoras, por lo que en las variables predictoras estamos trabajando con variables categóricas y continuas, como se pide en el enunciado de la práctica.

In [35]:
arbol = C45()
arbol.max_depth = 1
arbol.metodo = 2            # GINI
arbol.useEncoder = True     # Variable objetivo con 0s y 1s
arbol.regresion = False
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	72.0: Class value: 0	Counts: 105152
	72.0: Class value: 1	Counts: 13889


In [36]:
arbol = C45()
arbol.max_depth = 1
arbol.metodo = 2            # GINI
arbol.useEncoder = False    # Variable objetivo con 'No' y 'Yes'
arbol.regresion = False
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	72.0: Class value: 0	Counts: 105152
	72.0: Class value: 0	Counts: 13889


La `regresión` se trata de predecir un valor numérico en función de sus variables predictoras. La estructura generadad del árbol es igual pero en las hojas, en vez de tener una variable, tenemos un valor numérico. En nuestro caso este valor numérico rondará entre cero y uno, suponiendo 0 cuando no va a llover y 1 cuando sí va a llover. En base a esto podemos tomar la probabilidad de lluvia o no lluvia del valor numérico de la hoja. Este valor numérico se obtiene haciendo la media según el punton de corte en la hoja. Las dos celdas siguientes muestran dos ejemplos, el primero con `GINI` y `regresión` activada, y el segundo con `entropía` y `regresión` activada. La regresión nos permite ver de forma más sencilla el poder predecir si mañana va a llover o no. En la primera celda, tenemos que ramificando por `Humidity3pm`, en una hoja tenemos una probabilidad de que llueva del 0.146 y en la otra hoja una probabilidad de que llueva del 0.622.

In [37]:
arbol = C45()
arbol.max_depth = 1
arbol.metodo = 2            # GINI
arbol.useEncoder = True     # Variable objetivo con 0s y 1s
arbol.regresion = True
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	72.0: Valor Regresion = 0.14607763521195383
	72.0: Valor Regresion = 0.6222670250896057


In [38]:
arbol = C45()
arbol.max_depth = 2
arbol.metodo = 1            # Entropía
arbol.useEncoder = True     # Variable objetivo con 0 y 1
arbol.regresion = True
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	60.0: Feature Rainfall
		0.1: Valor Regresion = 0.08077221790746703
		0.1: Valor Regresion = 0.1855168386217637
	60.0: Feature Rainfall
		0.1: Valor Regresion = 0.3116082161710134
		0.1: Valor Regresion = 0.5370383348517765


Ahora vamos a calcular los mismos árboles que hemos calculado anteriormente pero en vez de usar `GINI` para calcular la ganancia de información, esta vez vamos a usar la `entropía`. Para ello diferenciamos esto usando la variable `metodo`, en la que si toma el valor de `2` usaría GINI y si toma el valor de `1` usaría la entropía.

Como se puede observar en las dos siguientes ejecuciones del algoritmo C45, usando la ganancia de entropía obtenemos que también vamos a partir el árbol por la misma variable predictora `Humidity3pm`, pero en este caso el punto de corte no es el mismo que usando `GINI`. En las ejecuciones anteriores vemos que el punto de corte era `72.0`, mientras que ahora es `60.0`. También podemos ver que usando el encoder la ejecución va mucho más rápida, como hemos visto en las dos ejecuciones anteriores.

In [39]:
arbol = C45()
arbol.max_depth = 1
arbol.metodo = 1            # Entropía
arbol.useEncoder = True     # Variable objetivo con 0 y 1
arbol.regresion = False
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	60.0: Class value: 0	Counts: 87200
	60.0: Class value: 0	Counts: 26383


In [40]:
arbol = C45()
arbol.max_depth = 1
arbol.metodo = 1            # Entropía
arbol.useEncoder = False    # Variable objetivo con 'No' y 'Yes'
arbol.regresion = False
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	60.0: Class value: 0	Counts: 87200
	60.0: Class value: 0	Counts: 26383


Una vez hemos explicado como hemos implementado las distintas funciones del algoritmo y las diferencias que hay, vamos a hacer ejecuciones pero esta vez con más profundidad en el árbol, para ver qué partición hace esta vez.

En el siguiente árbol usamos el `encoder` para acelerar la ejecución y establecemos la `profundidad` del árbol en `2`. El árbol incialmente hace una partición por `Humidity3pm` en el punto de corte de `60.0`, y en cada partición hace otra división por `Rainfall`, con punto de corte igual a `0.1`. Es importante comentar que el método de `ganancia de información` que estamos usando es la `entropía`.

In [41]:
arbol = C45()
arbol.max_depth = 2
arbol.metodo = 1            # Entropía
arbol.useEncoder = True     # Variable objetivo con 0 y 1
arbol.regresion = False
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	60.0: Feature Rainfall
		0.1: Class value: 0	Counts: 66280
		0.1: Class value: 0	Counts: 20920
	60.0: Feature Rainfall
		0.1: Class value: 0	Counts: 13171
		0.1: Class value: 1	Counts: 15326


Si en vez de profundidad `2` establecemos profundidad `3` vemos como las particiones iniciales son las mismas que el árbol anterior, pero luego divide por `Sunshine` y `WindGustSpeed` en la izquierda y por `Cloud3pm` y `WindGustSpeed` por la derecha.

In [42]:
arbol = C45()
arbol.max_depth = 3
arbol.metodo = 1            # Entropía
arbol.useEncoder = True     # Variable objetivo con 0 y 1
arbol.regresion = False
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	60.0: Feature Rainfall
		0.1: Feature Sunshine
			7.4: Class value: 0	Counts: 6486
			7.4: Class value: 0	Counts: 29190
		0.1: Feature WindGustSpeed
			40.03523007167319: Class value: 0	Counts: 12039
			40.03523007167319: Class value: 0	Counts: 8881
	60.0: Feature Rainfall
		0.1: Feature Cloud3pm
			7.0: Class value: 0	Counts: 11721
			7.0: Class value: 1	Counts: 1610
		0.1: Feature WindGustSpeed
			41.0: Class value: 0	Counts: 9294
			41.0: Class value: 1	Counts: 8006


Por último, vamos a ver un árbol de profundidad 3 usando la regresión.

In [43]:
arbol = C45()
arbol.max_depth = 3
arbol.metodo = 1            # Entropía
arbol.useEncoder = True     # Variable objetivo con 0 y 1
arbol.regresion = True
arbol.fit(X,y)
print(arbol.tree)

Feature Humidity3pm
	60.0: Feature Rainfall
		0.1: Feature Sunshine
			7.4: Valor Regresion = 0.20989158240955053
			7.4: Valor Regresion = 0.04828665514655539
		0.1: Feature WindGustSpeed
			40.03523007167319: Valor Regresion = 0.12887120115774242
			40.03523007167319: Valor Regresion = 0.25149599662874
	60.0: Feature Rainfall
		0.1: Feature Cloud3pm
			7.0: Valor Regresion = 0.27076463634666836
			7.0: Valor Regresion = 0.5261437908496732
		0.1: Feature WindGustSpeed
			41.0: Valor Regresion = 0.44059227157818703
			41.0: Valor Regresion = 0.6714189869171419
